# Libraries

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

In [26]:
user = "ALESSANDRO"

import external functions

In [27]:
%run functions.ipynb

# Retrieve all years links

In [30]:
# webpage to scrape
url = "https://www.daviscup.com/en/draws-results/historic-format/world-group.aspx"

# Path to the ChromeDriver
chrome_driver_path = f"C:/Users/{user}/Downloads/chromedriver.exe"

# Initialize the WebDriver
driver = webdriver.Chrome(executable_path=chrome_driver_path)
driver.get(url)
driver.maximize_window()
wait = WebDriverWait(driver, 15)

# Click on Accept All Cookies button
acceptCookie_Btn = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
driver.execute_script("arguments[0].click();", acceptCookie_Btn)

# Initialize an empty DataFrame to store all the data
final_df = pd.DataFrame()

# Loop through selected years
for year in range(2018,2017 , -1): 
    # Click on the dropdown arrow
    wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@class='selected']//div[@class='arrow']"))).click()
    time.sleep(4)  # Introduce a delay before clicking the element
    year_xpath = f"//a[text()='{year}']"
    print(year_xpath)
    
    try:
        # Locate the element for the specific year and click it
        year_element = wait.until(EC.presence_of_element_located((By.XPATH, year_xpath)))
        print(year_element)        
        # Click on the element
        year_element.click()
        
        # Wait for the page to load
        driver.implicitly_wait(10)  # You can adjust the waiting time as needed
        time.sleep(10)

        # Find all links with class "tie-link" within the tables
        tie_links = driver.find_elements(By.CSS_SELECTOR, "table.tie.ng-scope a.tie-link")

        # Initialize a list to store the extracted links
        links = []

        # Extract the links
        for link in tie_links:
            href = link.get_attribute("href")
            links.append(href)

        # Call the function to scrape and transform the data
        matches_df, players_df = to_df(links)
        # Cleaning DataFrames
        cleaned_matches_df = clean_matches_df(matches_df)
        cleaned_players_df = clean_players_df(players_df)

        # Merging DataFrames
        merged_df = merge_data_frames(cleaned_matches_df, cleaned_players_df)
        # Add a column with the respective year information
        merged_df['Year'] = year

        # Append data to the final DataFrame
#         final_df = pd.concat([final_df, merged_df], ignore_index=True)
        # Save the current year's data to a separate Excel file
        year_file_name = f"davis_{year}.xlsx"
        merged_df.to_excel(year_file_name, index=False)

    except TimeoutException:
        print(f"TimeoutException occurred while locating element for year {year}. Skipping...")
        continue  # Skip to the next iteration if element not found within the timeout
        
# Save the final DataFrame to an Excel file
#final_df.to_excel("old_system.xlsx", index=False)


C:\Users\ALESSANDRO\AppData\Local\Temp\ipykernel_11700\1435060639.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


//a[text()='2018']
<selenium.webdriver.remote.webelement.WebElement (session="80624790c59b61a69eaf9781cf831d47", element="90989FC5025F1A43D960F34864A6AEE5_element_95")>


C:\Users\ALESSANDRO\AppData\Local\Temp\ipykernel_11700\2024397676.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=f"C:/Users/{user}/Downloads/chromedriver.exe", options=chrome_options)


match: https://www.daviscup.com/en/draws-results/tie.aspx?id=M-DC-2018-WG-M-FRA-NED-01
Initializing Selenium...
Navigating to the webpage...
Waiting for the page to load...
['6', '4']
['3']
['3']
['7', '7']
['6']
['6']
['6']
['7', '7']
['3']
['7']
['4']
['6', '3']
['6']
['5']
['7', '8']
['6']
['6', '3']
['7', '7']
['6', '6']
['3']
['7', '7']
['6', '2']
['4']
['7', '7']
['7']
['6', '2']
['7']
['6']
['6', '5']
['5']
['7', '7']
['5']
Skipping match 5
Skipping match 5
matches df downloaded
players df downloaded
match: https://www.daviscup.com/en/draws-results/tie.aspx?id=M-DC-2018-WG-M-ITA-JPN-01
Initializing Selenium...
Navigating to the webpage...
Waiting for the page to load...
['4']
['6']
['6']
['3']
['2']
['6']
['3']
['4']
['6']
['6']
['4']
['6']
['6']
['4']
['7', '7']
['6']
['2']
['4']
['6']
['6', '1']
['5']
['7', '7']
['6', '3']
['5']
['7']
['6', '4']
['7', '7']
['7']
['6']
['1']
['6']
['6', '6']
['5']
['3']
['6']
['3']
['7', '8']
['7']
Skipping match 5
Skipping match 5
matches df d

In a year I have 15 matches (number of link), each match has a max number of 5 matches and on avg 4 matches. The number of single matches should be therefore between 60 and 70. We have two observations (one for each player or team) for each match for a total of 120/140 rows (max 75*2 = 150).

We have 5 players in each team, two team in each match, and 15 matches for a total of (5 * 2 * 15) 150 players/teams.

players of teams advancing to the next stage appear more than ones with the same information. it would be efficient to not repeat the data collection for those observations.